In [ ]:
# 单节点
# 使用 metadata 中的字段拆分 collection
# 图索引
# 稀疏索引
# 3万条数据指一个 collection 中3万个 points
# 混合检索
# 包括向量相似度（重要）+ metadata 中的字段
# bge
# bm42
# 主要看的是检索效率

In [ ]:
# cosine = normalize + dot

In [ ]:
# # case_0
# # 200字
# # case_1
# # 2000字

# # case_2
# # 20000字
# name = 'case_2'
# client.create_collection(
#     collection_name=name,
#     vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
    
#     # 在这里修改建库参数
# )
# # 不知道为啥1405个点落进去成118个了
# # 可能是一次性上传时有上限
# # 改为 for 循环一条一条上传
# print(f'final_points的长度是{len(final_points)}')
# i = 0
# for p in final_points:
#     client.upsert(
#         collection_name=name,
#         wait=False,
#         points=[p]
#     )
#     i += 1
# print(f'循环的次数是{i}')
# count_info = client.count(collection_name='case_2', exact=True)
# print(f'点的个数是{count_info.count}')
# # 看一下是否成功落库 'completed'
# # print(operation_info)

# # 最终发现是录入的文本重复段落过多，导致覆盖
# # 换源

In [ ]:
# search
# 主要聚焦于基于向量相似度的搜索操作
# query_points
# 参数类型更加多样化，可以预取，可以指定查找向量的位置

In [ ]:
# query_points 参数
import Para_test.emb as emb
from qdrant_client import QdrantClient, models
import time

def main():
    client = QdrantClient(url="")  

    # 要查询的集合名称
    collection_name = "case_1"  

    # 查询的内容，这里是密集向量 List[float]
    # 如果输入其他格式内容
    # str: 字符串
    # int: ID
    # None: 前 limit 个点

    text = '霎时间过了二百里水面'
    vector = emb.txt_to_vec(text)
    query = vector

    # 指定要使用的向量名称，如果为 None 则使用默认向量
    using = None

    # 预取查询
    # prefetch = models.Prefetch(
    #     queries=[
    #         models.Query(
    #             vector=[0.1, 0.2, 0.3],  # 预取的查询向量
    #             filter=models.Filter(
    #                 must=[
    #                     models.FieldCondition(
    #                         key="category",  # 预取过滤条件的字段
    #                         match=models.MatchValue(value="books")  # 预取过滤条件的值
    #                     )
    #                 ]
    #             )
    #         )
    #     ]
    # )

    # 过滤
    # 必须满足
    # query_filter = models.Filter(
    #                 must=[
    #                     models.FieldCondition(
    #                         key="category",
    #                         match=models.MatchValue(value="books")
    #                     )
    #                 ]
    # 满足其一
    # should=[
    #     models.FieldCondition(
    #         key="rating",
    #         range=models.Range(
    #             gte=4.0
    #         )
    #     )
    # ],
    # 必须不满足
    # must_not=[
    #     models.FieldCondition(
    #         key="status",
    #         match=models.MatchValue(value="out_of_stock")
    #     )
    # ]
    # )

    # 搜索参数
    """
    调优
    """
    search_params = models.SearchParams(
        hnsw_ef=128,  # 数值越大越精确
        exact=True,  # 是否使用精确搜索，false 会使用一些近似算法
        quantization = None,
        indexed_only = False
    )

    # 限制结果的数量
    limit = 3

    # 偏移量
    offset = 0

    # 是否返回 payload
    with_payload = True  

    # 是否返回 vector
    with_vectors = False  

    # 最小分数阈值
    score_threshold = 0.5  

    # 定义查找向量的位置，默认使用当前集合
    lookup_from = None  

    # 读取一致性
    # 只有在这些被查询的副本中都存在的值才会作为结果返回
    # 'int''majority''quorum''all'
    consistency = 'all'

    # 指定查询哪些分片
    shard_key_selector = None  

    # 超时单位为秒
    timeout = 30  




    # 执行100次查询操作
    start_time = time.time()
    for _ in range(100):
        results = client.query_points(
            collection_name=collection_name,
            query=query,
            using=using,
            # prefetch=prefetch,
            # query_filter=query_filter,
            search_params=search_params,
            limit=limit,
            offset=offset,
            with_payload=with_payload,
            with_vectors=with_vectors,
            score_threshold=score_threshold,
            lookup_from=lookup_from,
            consistency=consistency,
            shard_key_selector=shard_key_selector,
            timeout=timeout
        )
    end_time = time.time()
    for point in results.points:
        print(point.payload.get('page_content'))
        print(f'分数：{point.score:.4f}')

    print('\n')
    print(f'100次检索用时 {end_time-start_time:.4f}秒')
main()

In [ ]:
# ，霎时间过了二百里水面
# 分数：0.9743
# ，有二百里水面
# 分数：0.8851
# 。忽行至西洋大海
# 分数：0.7024


# 100次检索用时 5.0703秒

In [ ]:
# Pre-filtering
# 先处理metadata
# Post-filtering
# 先处理近似向量

In [ ]:
# 建库参数
from qdrant_client.models import Distance, models

def main():
    client = QdrantClient(url="")  

    # 要创建的集合名称
    collection_name = "case_2"  

    # 向量存储的配置
    vectors_config = models.VectorParams(size=1024, distance=Distance.COSINE)

    # 稀疏向量存储的配置
    sparse_vectors_config = None  

    # 分片数量
    shard_number = None

    # 分片方法
    sharding_method = None

    # 复制因子，在分布式模式下使用
    replication_factor = None

    # 写一致性因子，在分布式模式下使用
    write_consistency_factor = 1  

    # 是否将有效负载存储在磁盘上
    # True 时略微增加响应时间
    on_disk_payload = False  

    # HNSW 索引配置
    hnsw_config = models.HnswConfigDiff(
        m=16,  # 每个节点的边数越多会增加空间时间
        ef_construct=200  # 邻居越多时间越长
    )

    # 优化器配置
    optimizers_config = models.OptimizersConfigDiff(
        deleted_threshold=0.2,  # 设置删除阈值
        vacuum_min_vector_number=1000  # 设置清理的最小向量数量
    )

    # 预写日志配置
    wal_config = models.WalConfigDiff(
        wal_capacity_mb=32,
        wal_segments_ahead=0
    )

    # 量化配置
    quantization_config = None  

    # 从其他集合初始化
    init_from = None  

    # 超时时间
    timeout = 30  

    # 执行创建集合操作
    client.create_collection(
        collection_name=collection_name,
        vectors_config=vectors_config,
        sparse_vectors_config=sparse_vectors_config,
        shard_number=shard_number,
        sharding_method=sharding_method,
        replication_factor=replication_factor,
        write_consistency_factor=write_consistency_factor,
        on_disk_payload=on_disk_payload,
        hnsw_config=hnsw_config,
        optimizers_config=optimizers_config,
        wal_config=wal_config,
        quantization_config=quantization_config,
        init_from=init_from,
        timeout=timeout
    )
main()

In [ ]:
# 提交笔记后指出切分的小块短文本映射到1024维可能会导致稀疏现象

In [ ]:
# case_8的indexed_vectors_count不为0
# 猜测原因为indexing_threshold为800（default=20000)
# 导致系统在数据量还相对较小的时候就开始启用向量索引(HNSW)

# 注意到indexed_vectors_count=0时搜索Precision为1，如果indexed_vectors_count不为0则Precision会降低（case_8)
# 猜测原因为一旦启用了索引，搜索操作可能会使用近似算法来提高搜索速度，从而降低精度